# Flask를 활용한 매포

In [1]:
# !pip install Flask==2.0.1 torchvision==0.10.0

     |████████████████████████████████| 94 kB 2.3 MB/s 
     |████████████████████████████████| 22.1 MB 1.8 MB/s 
     |████████████████████████████████| 133 kB 45.3 MB/s 
     |████████████████████████████████| 288 kB 37.6 MB/s 
     |████████████████████████████████| 831.4 MB 2.6 kB/s 
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 1.0.1
    Uninstalling Werkzeug-1.0.1:
      Successfully uninstalled Werkzeug-1.0.1
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
  Attempting uninstall: Jinja2
    Found existing installation: Jinja2 2.11.3
    Uninstalling Jinja2-2.11.3:
      Successfully uninstalled Jinja2-2.11.3
  Attempting uninstall: itsdangerous
    Found existing installation: itsdangerous 1.1.0
    Uninstalling itsdangerous-1.1.0:
      Successfully uninstalled itsdangerous-1.1.0
  Attempting uninstall: torchvision
    Found exist

In [1]:
from flask import Flask
app = Flask(__name__)


@app.route('/')
def hello():
    return 'Hello World!'

In [4]:
# app.py
!FLASK_ENV=development FLASK_APP=app.py flask run

 * Serving Flask app 'app.py' (lazy loading)
 * Environment: development
 * Debug mode: on
Usage: flask run [OPTIONS]

Error: Could not import 'app'.


In [ ]:
from flask import Flask, jsonify
app = Flask(__name__)


# app.py에 predict 넣음
@app.route('/predict', methods=['POST'])
def predict():
    return jsonify({'class_id': 'IMAGE_NET_XXX', 'class_name': 'Cat'})

# image 준비

In [5]:
import io

import torchvision.transforms as transforms
from PIL import Image

def transform_image(image_bytes):
    my_transforms = transforms.Compose([transforms.Resize(255),
                                        transforms.CenterCrop(224),
                                        transforms.ToTensor(),
                                        transforms.Normalize(
                                            [0.485, 0.456, 0.406],
                                            [0.229, 0.224, 0.225])])
    image = Image.open(io.BytesIO(image_bytes))
    return my_transforms(image).unsqueeze(0)

In [6]:
# 이미지 바꿈
with open("../_static/img/sample_file.jpeg", 'rb') as f:
    image_bytes = f.read()
    tensor = transform_image(image_bytes=image_bytes)
    print(tensor)

FileNotFoundError: ignored

# 예측

In [ ]:
from torchvision import models

model = models.densenet121(pretrained=True)
model.eval()


def get_prediction(image_bytes):
    tensor = transform_image(image_bytes=image_bytes)
    outputs = model.forward(tensor)
    _, y_hat = outputs.max(1)
    return y_hat

In [ ]:
# Yhat은 class label을 보여주기에 이름으로 바꾸어야함
import json

imagenet_class_index = json.load(open('../_static/imagenet_class_index.json'))

def get_prediction(image_bytes):
    tensor = transform_image(image_bytes=image_bytes)
    outputs = model.forward(tensor)
    _, y_hat = outputs.max(1)
    predicted_idx = str(y_hat.item())
    return imagenet_class_index[predicted_idx]

In [ ]:
with open("../_static/img/sample_file.jpeg", 'rb') as f:
    image_bytes = f.read()
    print(get_prediction(image_bytes=image_bytes))

# API 서버와 model통합

In [7]:
from flask import request

@app.route('/predict', methods=['POST'])
def predict():
    if request.method == 'POST':
        file = request.files['file'] # request해서 파일 불러옴
        img_bytes = file.read() # 파일을 바이트로 바꿈
        class_id, class_name = get_prediction(image_bytes=img_bytes)
        return jsonify({'class_id': class_id, 'class_name': class_name})

In [8]:
import io
import json

from torchvision import models
import torchvision.transforms as transforms
from PIL import Image
from flask import Flask, jsonify, request


app = Flask(__name__)
imagenet_class_index = json.load(open('<PATH/TO/.json/FILE>/imagenet_class_index.json')) # josn load
model = models.densenet121(pretrained=True)
model.eval()


def transform_image(image_bytes):
    my_transforms = transforms.Compose([transforms.Resize(255),
                                        transforms.CenterCrop(224),
                                        transforms.ToTensor(),
                                        transforms.Normalize(
                                            [0.485, 0.456, 0.406],
                                            [0.229, 0.224, 0.225])])
    image = Image.open(io.BytesIO(image_bytes))
    return my_transforms(image).unsqueeze(0)


def get_prediction(image_bytes):
    tensor = transform_image(image_bytes=image_bytes)
    outputs = model.forward(tensor)
    _, y_hat = outputs.max(1)
    predicted_idx = str(y_hat.item())
    return imagenet_class_index[predicted_idx]


@app.route('/predict', methods=['POST'])
def predict():
    if request.method == 'POST':
        file = request.files['file']
        img_bytes = file.read()
        class_id, class_name = get_prediction(image_bytes=img_bytes)
        return jsonify({'class_id': class_id, 'class_name': class_name})


if __name__ == '__main__':
    app.run()

FileNotFoundError: ignored

### upgrad 방향

- /predict : request에 항상 이미지 파일이 있다고 가정

- 사용자들이 이미지 이외의 것을 보낼 수 있기에 예외 처리
- 모델이 많은 수의 이미지를 인식할 지라도 모든 이미지를 인식하는 것은 아님. 그러므로, 이를 방지할 필요있다
- UI 추가 가능